In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark_session = SparkSession.builder\
        .master("spark://192.168.2.250:7077") \
        .appName("PatrickHennig_Lab3_B")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

spark_session.sparkContext.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/20 09:08:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 52128)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 295, in handle
  

B.1 Load the CSV file from HDFS, and call show() to verify the data is loaded correctly.

B.2 Print the schema for the DataFrame.

B.3 Count the number of rows in the CSV file.

B.4 Count the number of partitions in the underlying RDD.

B.5 Drop the columns VIN, Latitude and Longitude.

B.6 Find the maximum fine amount. How many fines have this amount? You need to convert the ‘fine amount’ column to a float to do this correctly.

B.7 Show the top 20 most frequent vehicle makes, and their frequencies.

B.8 Let’s expand some abbreviations in the color column. Create a User Defined Function to
create a new column, ‘color long’, mapping the original colors to their corresponding values
in the dictionary below. If there is no key matching the original color, use the original color.
```
COLORS = {
'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White', 'WH':'White',
'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
}
```

B.9 Using this new column, what’s the most frequent colour value for Toyotas (TOYT)?


In [15]:
# B.1 - load file and .show
laparkcit=spark_session.read.format("csv").option("header", "true").load("hdfs://192.168.2.250:9000/parking-citations.csv")
laparkcit.show

# B.2 - print schema
laparkcit.printSchema()

# B.3 - count rows in csv
num_rows=laparkcit.count()
print("number of rows in dataframe:",num_rows)

# B.4 - count number of partitions in rdd
print("number of partitions in dataframe:",laparkcit.rdd.getNumPartitions())

# B.5 - drop columns Vin, Latitude, Longitude
laparkcit.drop("VIN","Latitude","Longitude")

# B.6 - convert fine column from str to float, get max fine amount, count fines with max amount
colname="Fine amount"
laparkcit = laparkcit.withColumn(colname, F.col(colname).cast("float"))
max_fine_amount=laparkcit.agg(F.max(F.col(colname))).collect()[0][f"max({colname})"]
print("max fine amount:",max_fine_amount)
fines_with_max_amount=laparkcit.filter(F.col(colname)==max_fine_amount)
print("num fines issued with max amount:",fines_with_max_amount.count())

# B.7 - calc car make frequencies, show top 20 makes (incl. freq)
count_colname="count"
freq_colname="frequency"
laparkcit \
    .groupBy("Make") \
    .agg(F.count("*").alias(count_colname)) \
    .withColumn(freq_colname,F.col(count_colname)/num_rows) \
    .orderBy(F.col(count_colname).desc()) \
    .drop(count_colname) \
    .limit(20) \
    .show()

# B.8 - expand car color names using udf
def expand_colorname(name_short:str)->str:
    COLORS = {
        'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
        'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
        'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
        'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
        'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
        'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
        'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White', 'WH':'White',
        'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
    }
    return COLORS.get(name_short,name_short)
    
expand_colorname_udf=F.udf(expand_colorname,"string")

color_long_colname="color long"
laparkcit=laparkcit.withColumn(color_long_colname,expand_colorname_udf("color"))

# B.9 - get most frequent color of Make==TOYT cars
color_count_colname="count"
laparkcit \
    .filter(F.col("Make")=="TOYT") \
    .groupBy(color_long_colname) \
    .agg(F.count("*").alias(color_count_colname)) \
    .orderBy(F.col(color_count_colname).desc()) \
    .limit(1) \
    .show()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



number of rows in dataframe: 13077724
number of partitions in dataframe: 16


max fine amount: 1100.0


num fines issued with max amount: 626


+----+--------------------+
|Make|           frequency|
+----+--------------------+
|TOYT|  0.1644604214005434|
|HOND| 0.11316923342318587|
|FORD|  0.0853539193823023|
|NISS| 0.07227045011807864|
|CHEV|  0.0682592781435057|
| BMW| 0.04611597553213388|
|MERZ| 0.04154377321313709|
|VOLK| 0.03303556490410717|
|HYUN|0.030962344823915845|
|DODG|0.029950624435872788|
|LEXS|0.028171568691922232|
| KIA| 0.02509266903017681|
|JEEP|0.024186165727308515|
|AUDI|0.019529009787941694|
|MAZD| 0.01853105326278487|
|OTHR| 0.01571726089340928|
| GMC|0.014137704695404185|
|INFI|0.013329154216742913|
|CHRY| 0.01223056856070674|
|SUBA|0.011824687537372711|
+----+--------------------+



+----------+------+
|color long| count|
+----------+------+
|      Gray|489697|
+----------+------+

